In [1]:
import findspark

In [2]:
findspark.init()

In [3]:
import pyspark

In [4]:
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession.builder \
        .master("local[*]") \
        .appName('test_rdd') \
        .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/10/05 08:32:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [7]:
df_green = spark.read.parquet('data/pq/green/*/*')

SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2


In [10]:
df_green.take(10)

[Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 18, 14, 54, 39), lpep_dropoff_datetime=datetime.datetime(2020, 1, 18, 15, 16, 34), store_and_fwd_flag='N', RatecodeID=1, PULocationID=66, DOLocationID=161, passenger_count=1, trip_distance=6.95, fare_amount=23.0, extra=0.0, mta_tax=0.5, tip_amount=5.31, tolls_amount=0.0, ehail_fee=None, improvement_surcharge=0.3, total_amount=31.86, payment_type=1, trip_type=1, congestion_surcharge=2.75),
 Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 21, 17, 55, 7), lpep_dropoff_datetime=datetime.datetime(2020, 1, 21, 18, 10, 20), store_and_fwd_flag='N', RatecodeID=1, PULocationID=74, DOLocationID=42, passenger_count=1, trip_distance=1.61, fare_amount=11.0, extra=1.0, mta_tax=0.5, tip_amount=0.0, tolls_amount=0.0, ehail_fee=None, improvement_surcharge=0.3, total_amount=12.8, payment_type=2, trip_type=1, congestion_surcharge=0.0),
 Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 24, 8, 15, 41), lpep_dropo

In [11]:
rdd = df_green \
      .select('lpep_pickup_datetime', 'PULocationID', 'total_amount') \
      .rdd

In [13]:
from datetime import datetime

In [14]:
start = datetime(year=2021, month=1, day=1)

# def filter_outliers(row):
#     return row['lpep_pickup_datetime'] >= start

In [15]:
# rows = rdd.take(10)
# row = rows[0]

In [19]:
# deal with the where clause
df_result = rdd \
    .filter(lambda row: row['lpep_pickup_datetime'] >= start) \
    .map(lambda row:(
    (row['lpep_pickup_datetime'].replace(minute=0, second=0, microsecond=0), row['PULocationID']),
    (row['total_amount'], 1)
    ))
    .reduceByKey(lambda left_value, right_value: )

In [22]:
# deal with the groupby
mapped_rdd = filtered_rdd.map(lambda row:(
    (row['lpep_pickup_datetime'].replace(minute=0, second=0, microsecond=0), row['PULocationID']),
    (row['total_amount'], 1)
))

In [26]:
df_green.show()

+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|VendorID|lpep_pickup_datetime|lpep_dropoff_datetime|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|
+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|       2| 2020-01-18 14:54:39|  2020-01-18 15:16:34|                 N|         1|          66|         161|              1|         6.95|       23.0|  0.0|    0.

In [31]:
columns = ['lpep_pickup_datetime', 'lpep_dropoff_datetime', 'PULocationID', 'DOLocationID', 'trip_distance']
duration_rdd = df_green \
    .select(columns) \
    .rdd

In [35]:
def apply_batch(partitions):
    return  [one]

In [36]:
rdd.mapPartitions(apply_batch).collect()

23/10/05 09:37:24 ERROR Executor: Exception in task 0.0 in stage 6.0 (TID 9)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 668, in main
    func, profiler, deserializer, serializer = read_command(pickleSer, infile)
                                               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 85, in read_command
    command = serializer._read_with_length(file)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 173, in _read_with_length
    return self.loads(obj)
           ^^^^^^^^^^^^^^^
  File "/opt/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 471, in loads
    return cloudpickle.loads(obj, encoding=encoding)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: code() argument 13 must be str, not int

	at org.apache.spark.api

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 6.0 failed 1 times, most recent failure: Lost task 0.0 in stage 6.0 (TID 9) (marrie executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 668, in main
    func, profiler, deserializer, serializer = read_command(pickleSer, infile)
                                               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 85, in read_command
    command = serializer._read_with_length(file)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 173, in _read_with_length
    return self.loads(obj)
           ^^^^^^^^^^^^^^^
  File "/opt/spark/python/lib/pyspark.zip/pyspark/seri